In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("manager_survey.xlsx")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28190 entries, 0 to 28189
Data columns (total 18 columns):
 #   Column                                                                                                                                                                                                                                Non-Null Count  Dtype         
---  ------                                                                                                                                                                                                                                --------------  -----         
 0   Timestamp                                                                                                                                                                                                                             28190 non-null  datetime64[ns]
 1   How old are you?                                                                        

Renombrar columnas

In [3]:
df = df.rename(columns={
    "Timestamp": "timestamp",
    "How old are you?": "age",
    "What industry do you work in?": "industry",
    "Job title": "job_title",
    "If your job title needs additional context, please clarify here:": "job_title_context",
    "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": "annual_salary",
    "How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.": "bonus_compensation",
    "Please indicate the currency": "currency",
    'If "Other," please indicate the currency here:': "currency_other",
    "If your income needs additional context, please provide it here:": "income_context",
    "What country do you work in?": "country",
    "If you're in the U.S., what state do you work in?": "state",
    "What city do you work in?": "city",
    "How many years of professional work experience do you have overall?": "years_experience_total",
    "How many years of professional work experience do you have in your field?": "years_experience_field",
    "What is your highest level of education completed?": "education_level",
    "What is your gender?": "gender",
    "What is your race? (Choose all that apply.)": "race"
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28190 entries, 0 to 28189
Data columns (total 18 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   timestamp                                        28190 non-null  datetime64[ns]
 1   age                                              28190 non-null  object        
 2   industry                                         28109 non-null  object        
 3   job_title                                        28188 non-null  object        
 4   job_title_context                                7289 non-null   object        
 5   annual_salary                                    28190 non-null  int64         
 6   bonus_compensation                               20834 non-null  float64       
 7   currency                                         28190 non-null  object        
 8   If "Other," please indicate the curr

In [4]:
df.country.value_counts()

country
United States     9029
USA               7958
US                2618
Canada            1574
United States      668
                  ... 
Contracts            1
Sri Lanka            1
Sri lanka            1
Danmark              1
Zambia               1
Name: count, Length: 390, dtype: int64

In [5]:
df['currency'].value_counts()

currency
USD        23473
CAD         1676
GBP         1594
EUR          654
AUD/NZD      505
Other        170
CHF           37
SEK           37
JPY           23
ZAR           17
HKD            4
Name: count, dtype: int64

Filtar sólo las filas en USD

In [6]:
df_usd = df[df['currency']=='USD'].copy()
df_usd.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23473 entries, 0 to 28189
Data columns (total 18 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   timestamp                                        23473 non-null  datetime64[ns]
 1   age                                              23473 non-null  object        
 2   industry                                         23410 non-null  object        
 3   job_title                                        23471 non-null  object        
 4   job_title_context                                6153 non-null   object        
 5   annual_salary                                    23473 non-null  int64         
 6   bonus_compensation                               17568 non-null  float64       
 7   currency                                         23473 non-null  object        
 8   If "Other," please indicate the currency 

In [9]:
countries = df_usd['country'].unique()
print(countries)

['United States' 'US' 'USA' 'usa' 'U.S.' 'United States ' 'us' 'Usa'
 'United States of America' 'United states' 'USA ' 'United states '
 'united states' 'United States of America ' 'U.S. ' 'U.S>' 'ISA' 'US '
 'United State' 'U.S.A' 'United Kingdom' 'Denmark' 'U.S.A.' 'America'
 'united states of america' 'Canada' 'Bermuda' 'Us' 'The United States'
 'United State of America' 'Mexico ' 'France' 'United Stated' 'u.s.'
 'UNITED STATES' 'united States' 'Kuwait' 'Sri lanka' 'Contracts'
 'USA-- Virgin Islands' 'United Statws' 'U.S'
 "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located"
 'Unites States ' 'Usa ' 'U.S.A. ' 'Japan' 'U. S. ' 'Germany'
 'United Sates' 'Japan ' 'United States of American ' 'Uniited States'
 'Worldwide (based in US but short term trips aroudn the world)'
 'United Sates of America' 'England '
 'United States (I 

Función para estandarizar paises

In [18]:
import re
import pandas as pd

def standardize_country(value):
    if pd.isna(value):
        return None

    v = str(value).strip().lower()

    # remove emojis & weird symbols
    v = re.sub(r"[^\w\s]", " ", v)
    v = re.sub(r"\s+", " ", v).strip()

    # --- UNITED STATES ---
    us_patterns = [
        r"\busa\b", r"\bus\b", r"\bu s\b", r"\bu s a\b",
        r"united states", r"united state", r"america",
        r"us of a", r"the us", r"the united states",
        r"unitedstates", r"United Sates", r"Unites States", r"United Statws", 
        r"Uniited States", r"Unted States", r"Untied States", r"Unite States"
    ]

    if any(re.search(p, v) for p in us_patterns):
        return "United States"

    # --- UNITED KINGDOM ---
    if v in ["uk", "u k", "united kingdom", "england", "scotland"]:
        return "United Kingdom"

    # --- UAE ---
    if v in ["uae", "u a e", "united arab emirates"]:
        return "United Arab Emirates"

    # --- COMMON CLEANUPS ---
    country_map = {
        "canada": "Canada",
        "mexico": "Mexico",
        "colombia": "Colombia",
        "germany": "Germany",
        "france": "France",
        "spain": "Spain",
        "italy": "Italy",
        "switzerland": "Switzerland",
        "netherlands": "Netherlands",
        "the netherlands": "Netherlands",
        "denmark": "Denmark",
        "danmark": "Denmark",
        "ireland": "Ireland",
        "poland": "Poland",
        "australia": "Australia",
        "new zealand": "New Zealand",
        "brazil": "Brazil",
        "argentina": "Argentina",
        "chile": "Chile",
        "uruguay": "Uruguay",
        "peru": "Peru",
        "panamá": "Panama",
        "panama": "Panama",
        "india": "India",
        "pakistan": "Pakistan",
        "bangladesh": "Bangladesh",
        "china": "China",
        "japan": "Japan",
        "south korea": "South Korea",
        "taiwan": "Taiwan",
        "philippines": "Philippines",
        "vietnam": "Vietnam",
        "singapore": "Singapore",
        "malaysia": "Malaysia",
        "thailand": "Thailand",
        "hong kong": "Hong Kong",
        "israel": "Israel",
        "nigeria": "Nigeria",
        "ghana": "Ghana",
        "kenya": "Kenya",
        "uganda": "Uganda",
        "morocco": "Morocco",
        "egypt": "Egypt",
        "south africa": "South Africa",
        "rwanda": "Rwanda",
        "zimbabwe": "Zimbabwe",
        "norway": "Norway",
        "sweden": "Sweden",
        "belgium": "Belgium",
        "russia": "Russia",
        "ukraine": "Ukraine",
        "qatar": "Qatar",
        "saudi arabia": "Saudi Arabia",
        "united arab emirates": "United Arab Emirates",
        "cuba": "Cuba",
        "romania": "Romania",
        "costa rica": "Costa Rica",
        "ecuador": "Ecuador",
        "jamaica": "Jamaica",
        "liberia": "Liberia",
        "somalia": "Somalia",
        "sierra leone": "Sierra Leone",
        "eritrea": "Eritrea",
        "burma": "Myanmar",
        "myanmar": "Myanmar",
    }

    if v in country_map:
        return country_map[v]

    # --- INVALID / NON-COUNTRY ANSWERS ---
    invalid_keywords = [
        "remote", "bonus", "commission", "salary", "company",
        "government", "worldwide", "international", "na"
    ]
    if any(k in v for k in invalid_keywords):
        return "Other_Invalid"

    # Default: title case cleaned value
    return v.title()

In [19]:
df_usd["country_standardized"] = df_usd["country"].apply(standardize_country)

In [20]:
countries = df_usd['country_standardized'].unique()
print(countries)

['United States' 'Isa' 'United Kingdom' 'Denmark' 'Canada' 'Bermuda'
 'Mexico' 'France' 'Kuwait' 'Sri Lanka' 'Contracts' 'United Statws'
 'Other_Invalid' 'Unites States' 'Japan' 'Germany' 'United Sates'
 'Uniited States' 'Colombia' 'Unted States' 'United Stattes' 'Sweden'
 'Cayman Islands' 'Uniyed States' 'Uniyes States' 'U A' 'Puerto Rico'
 'Rwanda' 'United Arab Emirates' 'U Sa' 'United Status' 'Russia' 'Uxz'
 'Ireland' 'Virginia' 'Uss' 'Uniteed States' 'Hartford' 'United Stares'
 'I S' 'Hong Kong' 'Unite States' 'China' 'Cambodia' 'Vietnam' 'Australia'
 'India' 'Bangladesh' 'Israel' 'Eritrea' 'Is' 'Cuba' 'Cote D Ivoire'
 'Somalia' 'Norway' 'Malaysia' 'Sierra Leone' 'Singapore' 'Italy'
 'The Bahamas' 'Costa Rica' 'United Statues' 'Argentina' 'Untied States'
 'Chile' 'Usab' 'Unitied States' 'Qatar' 'United Sttes' 'Nigeria' 'Panama'
 'Switzerland' 'Spain' 'Brazil' 'Uniter Statez' 'Uruguay' 'Pakistan'
 'New Zealand' 'Uganda' 'Philippines' 'Poland' 'Netherlands' 'Morocco'
 'Africa' 'Ecuad

Filtrar sólo las filas de USA

In [24]:
df_usa = df_usd[df_usd["country"]=="United States"].copy()
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 28187
Data columns (total 19 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   timestamp                                        9026 non-null   datetime64[ns]
 1   age                                              9026 non-null   object        
 2   industry                                         9012 non-null   object        
 3   job_title                                        9025 non-null   object        
 4   job_title_context                                2322 non-null   object        
 5   annual_salary                                    9026 non-null   int64         
 6   bonus_compensation                               6783 non-null   float64       
 7   currency                                         9026 non-null   object        
 8   If "Other," please indicate the currency h

In [36]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
cities = df_usa['city'].unique()
print(cities)


['Boston' 'Yuma' 'Palm Coast' ... 'Olathe ' 'Okemos' 'Livonia']


Función para limpiar las ciudades

In [37]:
import re
import pandas as pd

def clean_city(value):
    if pd.isna(value):
        return None

    v = str(value).strip()

    # lowercase for processing
    v_lower = v.lower()

    # drop obvious non-city answers
    invalid_keywords = [
        "remote", "worldwide", "international", "usa", "us", "united states",
        "canada", "uk", "government", "company", "overseas", "based",
        "from", "work", "home", "country", "state", "province"
    ]

    if any(k in v_lower for k in invalid_keywords):
        return None

    # remove emojis & special characters (keep letters, spaces, hyphens)
    v = re.sub(r"[^\w\s\-]", "", v)

    # normalize whitespace
    v = re.sub(r"\s+", " ", v).strip()

    # handle very short garbage values
    if len(v) < 2:
        return None

    # title case (San francisco -> San Francisco)
    v = v.title()

    return v


In [ ]:
df_usa["city_clean"] = df_usa["city"].apply(clean_city)

Función para convertir a pesos

In [30]:
def to_cop(amount, currency, usd_to_cop=3690.49):
    """
    Convert amount from given currency to Colombian Pesos (COP).
    - amount: numeric salary/compensation amount
    - currency: standardized currency label (e.g., 'USD', 'COP')
    - usd_to_cop: exchange rate 1 USD -> COP
    
    Returns converted amount in COP
    """
    if pd.isna(amount):
        return 0

    currency = str(currency).strip().upper()

    # Si está en COP ya
    if currency in ["COP", "COL", "PESO", "PESOS", "COP"]:
        return amount

    # Solo USD por ahora
    if currency in ["USD", "$", "US DOLLAR", "US"]:
        return amount * usd_to_cop

    # TODO: soportar más divisas (EUR, GBP, etc.)
    return amount * usd_to_cop  # fallback

TRM 08 FEBRERO 2026

In [32]:
usd_to_cop = 3690.49

In [33]:
df_usa["salario_anual_cop"] = df_usa.apply(
    lambda row: to_cop(row["annual_salary"], row["currency"], usd_to_cop),
    axis=1
)

df_usa["compensaciones_cop"] = df_usa.apply(
    lambda row: to_cop(row["bonus_compensation"], row["currency"], usd_to_cop),
    axis=1
)

df_usa["total_compensacion_cop"] = df_usa["salario_anual_cop"] + df_usa["compensaciones_cop"]

In [34]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9026 entries, 0 to 28187
Data columns (total 22 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   timestamp                                        9026 non-null   datetime64[ns]
 1   age                                              9026 non-null   object        
 2   industry                                         9012 non-null   object        
 3   job_title                                        9025 non-null   object        
 4   job_title_context                                2322 non-null   object        
 5   annual_salary                                    9026 non-null   int64         
 6   bonus_compensation                               6783 non-null   float64       
 7   currency                                         9026 non-null   object        
 8   If "Other," please indicate the currency h

In [35]:
df_usa.to_excel("manager_survey_cleaned.xlsx", index=False)